In [1]:
import torch

# 1.Veri Seti Oluşturma

Text : emma , mia


In [2]:
stoi = {'.': 0, 'a': 1, 'e': 2, 'i': 3, 'm': 4}

Context ve Target Oluşturma
- Kontekst boyutu 3 olsun (her bir giriş, 3 karakterlik bir kontekst içerir).
- Örnek veri için giriş (X) ve çıkış (Y) çiftlerini oluşturalım:
> emma için;
  - Kontekst: [., ., e] → Hedef: m → [0, 0, 2] → 4
  - Kontekst: [., e, m] → Hedef: m → [0, 2, 4] → 4
  - Kontekst: [e, m, m] → Hedef: a → [2, 4, 4] → 1
  - Kontekst: [m, m, a] → Hedef: . → [4, 4, 1] → 0

  > mia için;
  - Kontekst: [., ., m] → Hedef: i → [0, 0, 4] → 3
  - Kontekst: [., m, i] → Hedef: a → [0, 4, 3] → 1
  - Kontekst: [m, i, a] → Hedef: . → [4, 3, 1] → Ch0

In [3]:
X = torch.tensor([
    [0,0,2],   # . . e ---> m
    [0,2,4],   # . e m ---> m
    [2,4,4],    # e m m --- > a
    [4,4,1],   # m m a --- > .
    [0,0,4],   #. . m ---- > i
    [0,4,3],  # . m i -----> a
    [4,3,1]    # m i a -----> .
])

Y = torch.tensor([4,4,1,0,3,1,0] )  # Hedef karakterlerin indexleri

Mini Batch Seçimi

- Mini-batch boyutu 2 olsun.Rastgele seçilen indexler [0,4]
- Mini batch girş ve çıkışları:

In [4]:
Xb = X[[0,4]]
Yb = Y[[0,4]]
print(Xb,Yb,Xb.shape , Yb.shape)

tensor([[0, 0, 2],
        [0, 0, 4]]) tensor([4, 3]) torch.Size([2, 3]) torch.Size([2])


# 2.Kullanılacak PARAMETRELER

Model Parametreleri

- Modelin boyutlarını basitleştirelim
  1. Gömme vectör boyutu 2 ---> Her karaktere ait 2 boyut(özellik)
  2. Birinci katman nöron sayısı : 3 ----> n_hidden
  3. Karakter sayısı : 5 (. , a , e , i , m)---> vocab_size
  4. Context boyutu : 3 ----> İnput ----> Katmana girişi ---> Context boyutu x gömme vectör boyutu 3 * 2 = 6

In [5]:
# Parametreler

# Gömme tablosu 5 karakter ve her biri 2 boyutlu
C = torch.tensor([
    [1.0,  0.5],   # . (0) ---> özellikler boyutlar
    [0.2 , 0.8],  # a (1)
    [0.4 , 0.6] ,  # e (2)
    [0.7 , 0.3] ,  # i (3)
    [0.9 , 0.1]   # m (4)
], dtype=torch.float32)


# ----------1.KATMAN ------------

# Birinci katman ( 6 giriş , 3 nöron)

W1 = torch.tensor([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9],
    [1.0, 1.1, 1.2],
    [1.3, 1.4, 1.5],
    [1.6, 1.7, 1.8]
], dtype=torch.float32)

# Bias değerleri
b1 = torch.tensor([0.1,0.2,0.3] , dtype=torch.float32)

# ----------BATCHNORM--------------

bngain = torch.tensor([[1.0,1.0,1.0]] , dtype=torch.float32)
bnbias = torch.tensor([[0.0,0.0,0.0]] , dtype=torch.float32)


# -------------İKİNCİ KATMAN----------

# İkinci katman (3 nöron, 5 çıkış)
W2 = torch.tensor([
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.6, 0.5, 0.4, 0.3, 0.2],
    [0.7, 0.6, 0.5, 0.4, 0.3]
], dtype=torch.float32)

b2 = torch.tensor([0.1, 0.1, 0.1, 0.1, 0.1], dtype=torch.float32)

# 3.Forward Pass ( ileri Yayılım )

Şimdi mini-batch Xb = [[0,0,2],[0,0,4]] , Yb = [4,3] üzerinde ileri yayılımı adım adım hesaplayacağız

1. Adım : Gömme (Embedding)

- Her karakteri gömme vektörüne dönüştürelim

Xb[0] = [0 , 0 , 2] ----> ( .  .  e )

C[0] = [1.0 , 0.5] --->  .
C[0] = [1.0 , 0.5] --->  .
C[2] = [0.4 , 0.6] --->  e

Xb[1] = [0 , 0 , 4 ] ----> (. . m)

C[0] =[1.0, 5.0]  ---> .
C[0] =[1.0 , 0.5] ---> .
C[4] =[0.9 , 0.1] ---> m

In [6]:
emb = C[Xb]   # (2 , 3 ,2) 2 örnek 3 context 2 boyut
#emb[0] = [[1.0,0.5],[1.0,0.5],[0.4,0.6]]
#emb[1]  [[1.0,0.5],[1.0,0.5],[0.9,0.1]]

In [ ]:
emb

tensor([[[1.0000, 0.5000],
         [1.0000, 0.5000],
         [0.4000, 0.6000]],

        [[1.0000, 0.5000],
         [1.0000, 0.5000],
         [0.9000, 0.1000]]])

In [7]:
# BİRLEŞTİRME (FLATTEN)

embcat = emb.view(emb.shape[0],-1)  # (2,6) 2 örnek 2*3 = 6 boyut
# embcat[0] = [1.0,0.5,1.0,0.5,0.4,0.6]
# embcat[1] = [1.0,0.5,1.0,0.5,0.9,0.1]
embcat

tensor([[1.0000, 0.5000, 1.0000, 0.5000, 0.4000, 0.6000],
        [1.0000, 0.5000, 1.0000, 0.5000, 0.9000, 0.1000]])

# 4.Birinci Katman ( Linear )


hprebn = embcat @ W1 + b1

Matris Çarpımı : embcat (2,6)  @ (6,3) = (2,3)

Örnek Matris Çarpımı : embcat[0] = [1.0,0.5,1.0,0.5,0.4,0.6]

W1 = tensor([
        [0.1, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.7, 0.8, 0.9],
        [1.0, 1.1, 1.2],
        [1.3, 1.4, 1.5],
        [1.6, 1.7, 1.8]])

b1 = [0.1 , 0.2 , 0.3]

hprebn = [0,0] = 1.0*0.1+0.5*0.4+1.0*0.7+0.5*1.0+0.4*1.3+0.6*1.6 = 1.1+ 0.2 + 0.7 + 0.5 + 0.52 + 0.96 = 3.03
hprebn [0,1] = 1.0*0.2+0.5*0.5+1.0*0.8+0.5*1.1+0.4*1.4+0.6*1.7 = 0.2+0.25+0.8+ 0.55 + 0.56 + 1.02 = 3.38
hprebn[0,2] = 1.0*0.3 + 0.5*0.6 + 1.0*0.9 + 0.5*1.2 + 0.4*1.5+ 0.6*1.8 = 0.3 + 0.3 + 0.9 + 0.6 + 0.6 + 1.08 = 3.78

Bias Ekleme = hprebn[0] = [3.03 + 0.1 , 3.38 + 0.2 , 3.78 + 0.3] = [3.13 , 3.58  , 4.08]

embcat[1] = [1.0 , 0.5 , 1.0 , 0.5 , 0.9 , 0.1]

hprebn[1,0] = [1.0*0.1 + 0.5 * 0.4...] benzer şekilde hesaplanır
hprebn[1] = [3.08 , 3.48 , 3.88]



In [8]:
hprebn = torch.tensor([
    [3.13,3.58,4.08],
    [3.08,3.48,3.88]
] , dtype= torch.float32)

# 5.BATCHNORM

--- Girişleri Normalize etmek

## 1.Ortalama Almak

In [10]:
bnmeani = hprebn.mean(0 , keepdim = True)
# bnmeani[0] = (3.13 + 3.08) / 2  = 3.105
# bnmeani[1] = (3.58 + 3.48) / 2 = 3.53
# bnmeani[2] = (4.08 + 3.88) / 2 = 3.98
#bnmeani= tensor([[3.1050, 3.5300, 3.9800]])
bnmeani


tensor([[3.1050, 3.5300, 3.9800]])

## 2.Fark ve Kare

In [11]:
bndiff = hprebn - bnmeani
# bndiff[0] = [3.13 - 3.105 , 3.58 - 3.53 , 4.08 - 3.98] = [0.025 , 0.05 , 0.1]
# bndiff[1] = [3.08 - 3.105 , 3.48 - 3.53 , 3.88 - 3.98] = [-0.025 , -0.05, -0.1]

bndiff2 = bndiff**2
# bndiff2[0] = [0.000625,0.0025,0.01]
# bndiff2[1] = [0.000625,0.0025,0.01]
bndiff2

tensor([[0.0006, 0.0025, 0.0100],
        [0.0006, 0.0025, 0.0100]])

## 3.Varyans (Bessel Düzeltmesi)

--- Varyans  : bndiff2 nin ortalaması : bessel düzeltmesi ortalama alırken eleman sayısının bir eksiğine bölünür


In [ ]:
bnvar = 1 / (2-1) * bndiff2.mean(0 , keepdim=True)  # Sütun toplanır eleman sayısının bir eksiğine böleriz

#bnvar[0] = 1/1 (0.000625 + 0.000625) = 0.00125
# bnvar[1] = 1/1 (0.0025 + 0.0025) = 0.005
# bnvar[2] = 1/1(0.01 + 0.01) = 0.02

bnvar = [[0.00125,0.005,0.02]]
bnvar_inv = (bnvar + 1e-5) ** -0.5
# bnvar_inv[0] = (0.00125 + 1e - 5) ^ (-0.5) ≈ 28.28
# bnvar_inv[1] = (0.005 + 1e-5) ^(-0.5) ≈ 14.14
#bnvar_inv[2] = (0.02 + 1e - 5) ^(0.5) ≈ 7.07
bnvar_inv = [[28.28,14.14,7.07]]

## 4.Normalizasyon

In [ ]:
bnraw = bndiff * bnvar_inv
# bnraw[0] = [0.025 * 28.28 , 0.05*14.14 , 0.1 * 7.07] ≈ [0.707 , 0.707 , 0.707 ]
#bnraw[1] = [-0.025 * 28.28 , -0.05 * 14.14 , -0.1 * 7.07] ≈ [-0.707 ,-0.707 , -0.707]

## 5.Ölçek ve Öteleme


In [ ]:
hpreact = bngain * bnraw + bnbias
# bngain = [[1,1,1,]] , bnbias = [0,0,0]
# hpreact = bnraw

hpreact = torch.tensor([
    [0.707,0.707,0.707],
    [-0.707,-0.707,-0.707]
],dtype = torch.float32)

# 6.Tanh Aktivasyonu

In [ ]:
h = torch.tanh(hpreact)
# tanh(0.707) ≈ 0.614 , tanh(-0.707) ≈ -0.614

h = torch.tensor([
    [0.614,0.614,0.614],
    [-0.614,-0.614,-0.614]
], dtype = torch.float32)

# 7.İkinci Katman(Linear)

logits = h @ W2 + b2
Matris Çarpımı = h(2,3) @ W2(3,5) = (2,5)

Örnek1 =>
 h = [[0.614,0.614,0.614],
    [-0.614,-0.614,-0.614]]

W2 = torch.tensor([
    [0.5, 0.4, 0.3, 0.2, 0.1],
    [0.6, 0.5, 0.4, 0.3, 0.2],
    [0.7, 0.6, 0.5, 0.4, 0.3]
], dtype=torch.float32)

b2 = torch.tensor([0.1, 0.1, 0.1, 0.1, 0.1], dtype=torch.float32)

logits[0,0] = 0.614*0.5 + 0.614*0.6 + 0.614 * 0.7 + 0.1 = 0.307 + 0.3684 + 0.4298 + 0.1 = 1.2052
...
logits[0] = [1.2052 , 1.0828 , 0.9604 , 0.8380 , 0.7156]

Örnek 2 >
Aynı hesaplama [-0.614 * 0.5 + -0.614 * 0.6...

h[1] = [-1.2052 , -1.0828 , -0.9604 , -0.8380 , -0.7156]

In [ ]:
logits = torch.tensor([
    [1.2052, 1.0828, 0.9604, 0.8380, 0.7156],
    [-1.2052, -1.0828, -0.9604, -0.8380, -0.7156]
], dtype=torch.float32)

# 8.Kayıp Hesaplama(Cross - Entropy Loss)

## 1 . Sayısal kararlılık için Normalizasyon

In [ ]:
logit_maxes = logits.max(1 , keepdim = True).values
# logit_maxes[0] = 1.2052 , logit_maxes[1] = 0.7156

norm_logits = logits - logit_maxes

# norm_logits[0] = [0, -0.1224, -0.2448, -0.3672, -0.4896]
# norm_logits[1] = [-0.4896, -0.3672, -0.2448, -0.1224, 0]

## 2.Softmax

In [ ]:
counts = norm_logits.exp()
# counts[0] ≈ [1, 0.885, 0.783, 0.692, 0.613]
# counts[1] ≈ [0.613, 0.692, 0.783, 0.885, 1]

counts_sum = count.sum(1 , keepdim = True)
# counts_sum[0] ≈ 3.973, counts_sum[1] ≈ 3.973

count_sum_inv = counts_sum ** -1
# counts_sum_inv ≈ [0.252, 0.252]

probs = counts * counts_sum_inv
# probs[0] ≈ [0.252, 0.223, 0.197, 0.174, 0.154]
# probs[1] ≈ [0.154, 0.174, 0.197, 0.223, 0.252]

## 3.Log Olasılıklar

In [ ]:
logprobs =  probs.log()
# logprobs[0] ≈ [-1.378, -1.500, -1.623, -1.746, -1.869]
# logprobs[1] ≈ [-1.869, -1.746, -1.623, -1.500, -1.378]

## 4.Kayıp

- Yb = [4 ,3] (hedefler : m , i)
- Loss = - logprobs[range(2) , Yb].mean()
- logprobs[0,4] = -1.869 , logprobs[1,3] = -1.500
- loss = -(-1.869 + -1.500) / 2 = 1.6845

In [ ]:
loss = torch.tensor(1.6845)